In [48]:
import pandas as pd

In [49]:
df = pd.read_csv("combined_data.csv")

In [50]:
df.head()

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...


In [51]:
df.shape

(83448, 2)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83448 entries, 0 to 83447
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   83448 non-null  int64 
 1   text    83448 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [53]:
df.duplicated().sum()

np.int64(0)

In [54]:
df.isnull().sum()

,0
label,0
text,0


In [55]:
df['text'] = df['text'].str.lower()

In [56]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [57]:
def remove_punctuation(text):
    """
      Removes punctuation from the text using spaCy.
    """
    doc = nlp(text)
    cleaned = [token.text for token in doc if not token.is_punct]
    return " ".join(cleaned)

In [58]:
def remove_stopwords(text):
    """
      Removes stopwords from the text using spaCy.
    """
    doc = nlp(text)
    cleaned = [token.text for token in doc if not token.is_stop]
    return " ".join(cleaned)

In [59]:
def remove_numbers(text):
    """
    Removes numbers from the text using spaCy.
    """
    doc = nlp(text)
    cleaned = [token.text for token in doc if not token.like_num]
    return " ".join(cleaned)

In [60]:
%pip install emoji


In [61]:
import emoji
import re

def emojis_to_text(text):
    """
    Replaces emojis in the text with descriptive text.
    """
    emoji_text = emoji.demojize(text, language='en')
    return re.sub(r'[^\w\s]', '', emoji_text)

In [62]:
def simple_tokenize(text):
    return text.split()

df['tokens'] = df['text'].apply(simple_tokenize)

In [63]:
df.head()

,label,text,tokens
0,1,ounce feather bowl hummingbird opec moment ala...,"[ounce, feather, bowl, hummingbird, opec, mome..."
1,1,wulvob get your medircations online qnb ikud v...,"[wulvob, get, your, medircations, online, qnb,..."
2,0,computer connection from cnn com wednesday es...,"[computer, connection, from, cnn, com, wednesd..."
3,1,university degree obtain a prosperous future m...,"[university, degree, obtain, a, prosperous, fu..."
4,0,thanks for all your answers guys i know i shou...,"[thanks, for, all, your, answers, guys, i, kno..."


In [69]:
df['cleaned_text'] = df['tokens'].apply(lambda x: " ".join(x))

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [71]:
tfidf_vectorizer = TfidfVectorizer(
    max_features=80,      # Maximum number of words to consider
    ngram_range=(1,1)     # Use unigrams (single words)
)

In [72]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])

In [73]:
y = df['label']

In [74]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [75]:
x_train, x_val, y_train, y_val = train_test_split(tfidf_matrix, y, test_size=0.3)

In [79]:
from sklearn.neighbors import KNeighborsClassifier

In [80]:
knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)

KNeighborsClassifier()

In [81]:
y_pred = knn_model.predict(x_val)

In [82]:
from sklearn.metrics import accuracy_score, classification_report

In [83]:
print("\n", classification_report(y_val, y_pred))


               precision    recall  f1-score   support

           0       0.94      0.92      0.93     11886
           1       0.93      0.94      0.93     13149

    accuracy                           0.93     25035
   macro avg       0.93      0.93      0.93     25035
weighted avg       0.93      0.93      0.93     25035

